**IR_Assignment-4**

**Importing necessary Library**

In [53]:
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead

GPU check 

In [54]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

**Review dataset loading of Amozen**

In [55]:
review_df = pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")
review_df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


**printing the Text and Summary**

In [56]:
review_df['Text']

0         I have bought several of the Vitality canned d...
1         Product arrived labeled as Jumbo Salted Peanut...
2         This is a confection that has been around a fe...
3         If you are looking for the secret ingredient i...
4         Great taffy at a great price.  There was a wid...
                                ...                        
568449    Great for sesame chicken..this is a good if no...
568450    I'm disappointed with the flavor. The chocolat...
568451    These stars are small, so you can give 10-15 o...
568452    These are the BEST treats for training and rew...
568453    I am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

In [57]:
from bs4 import BeautifulSoup

def preprocess_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Apply text preprocessing to the 'Summary' column in review_df
review_df['Text'] = review_df['Text'].apply(preprocess_text)

# Print the preprocessed summaries
print(review_df['Text'])

/tmp/ipykernel_34/352302355.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price.  there was a wid...
                                ...                        
568449    great for sesame chicken..this is a good if no...
568450    i'm disappointed with the flavor. the chocolat...
568451    these stars are small, so you can give 10-15 o...
568452    these are the best treats for training and rew...
568453    i am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object


In [58]:
review_df['Summary']

0                      Good Quality Dog Food
1                          Not as Advertised
2                      "Delight" says it all
3                             Cough Medicine
4                                Great taffy
                         ...                
568449                   Will not do without
568450                          disappointed
568451              Perfect for our maltipoo
568452    Favorite Training and reward treat
568453                           Great Honey
Name: Summary, Length: 568454, dtype: object

**Data Preprocessing**

**Removing Null value in summary and Text**

In [59]:
review_df.dropna(subset=['Text', 'Summary'], inplace=True)

In [60]:
len(review_df)

568427

In [61]:
from bs4 import BeautifulSoup

def preprocess_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Apply text preprocessing to the 'Summary' column in review_df
review_df['Summary'] = review_df['Summary'].apply(preprocess_text)

# Print the preprocessed summaries
print(review_df['Summary'])

/tmp/ipykernel_34/293721340.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
/tmp/ipykernel_34/293721340.py:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


0                      good quality dog food
1                          not as advertised
2                      "delight" says it all
3                             cough medicine
4                                great taffy
                         ...                
568449                   will not do without
568450                          disappointed
568451              perfect for our maltipoo
568452    favorite training and reward treat
568453                           great honey
Name: Summary, Length: 568427, dtype: object


In [62]:
len(review_df)

568427

In [63]:
review_df['model_input'] = review_df['Text'] + " TL;DR " + review_df['Summary']
print(review_df['model_input'])

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price.  there was a wid...
                                ...                        
568449    great for sesame chicken..this is a good if no...
568450    i'm disappointed with the flavor. the chocolat...
568451    these stars are small, so you can give 10-15 o...
568452    these are the best treats for training and rew...
568453    i am very satisfied ,product is as advertised,...
Name: model_input, Length: 568427, dtype: object


In [64]:
review_df['model_input'].values[:6]

array(['i have bought several of the vitality canned dog food products and have found them all to be of good quality. the product looks more like a stew than a processed meat and it smells better. my labrador is finicky and she appreciates this product better than  most. TL;DR good quality dog food',
       'product arrived labeled as jumbo salted peanuts...the peanuts were actually small sized unsalted. not sure if this was an error or if the vendor intended to represent the product as "jumbo". TL;DR not as advertised',
       'this is a confection that has been around a few centuries.  it is a light, pillowy citrus gelatin with nuts - in this case filberts. and it is cut into tiny squares and then liberally coated with powdered sugar.  and it is a tiny mouthful of heaven.  not too chewy, and very flavorful.  i highly recommend this yummy treat.  if you are familiar with the story of c.s. lewis\' "the lion, the witch, and the wardrobe" - this is the treat that seduces edmund into sell

In [65]:
total_length = 0
total_reviews = 0

for review_df in review_df.model_input.values:
    total_length += len(review_df.split())
    total_reviews += 1

mean_length = total_length / total_reviews
mean_length

84.21211870653576

In [66]:
maximum_length = 180

In [67]:
review_df = review_df.sample(100)
review_df = review_df.model_input.values.tolist()
len(review_df)

AttributeError: 'str' object has no attribute 'sample'

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming review_df is a DataFrame with a 'model_input' column

# Sample 100 rows from the DataFrame
sampled_df = review_df.sample(100)

# Extract the 'model_input' values as a list
data = sampled_df['model_input'].values.tolist()

# Split the data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Check the lengths of the train and test sets
print("Train set size:", len(train_data))
print("Test set size:", len(test_data))

In [ ]:
print(review_df[1])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")

In [ ]:
model = model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
tokenizer.encode(" TL;DR ")

In [ ]:
extra_length = len(tokenizer.encode(" TL;DR "))

In [ ]:
class ReviewDataset(Dataset):  
    def __init__(self, tokenizer, reviews, max_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.reviews = reviews
        self.result = []

        for review in self.reviews:
            # Encode the text using tokenizer.encode(). We add EOS at the end
            tokenized = self.tokenizer.encode(review + self.eos)
            
            # Padding/truncating the encoded sequence to max_len 
            padded = self.pad_truncate(tokenized)            

            # Creating a tensor and adding to the result
            self.result.append(torch.tensor(padded))

    def __len__(self):
        return len(self.result)


    def __getitem__(self, item):
        return self.result[item]

    def pad_truncate(self, name):
        name_length = len(name) - extra_length
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id] 
        else:
            result = name
        return result

In [ ]:
dataset = ReviewDataset(tokenizer, review_df, maximum_length)

In [ ]:
print(len(dataset[0]))

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

In [ ]:
def train(model, optimizer, dataloader, epochs):    
    for epoch in range(1, epochs + 1):
        total_loss = 0.0
        for batch_idx, batch in enumerate(dataloader, start=1):
            optimizer.zero_grad()
            batch = batch.to(device)  # Assuming 'device' is defined elsewhere
            output = model(batch, labels=batch)
            loss = output[0]
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if batch_idx % 100 == 0:
                print(f"Epoch [{epoch}/{epochs}], Batch [{batch_idx}], Loss: {loss.item():.4f}")
        print(f"Epoch [{epoch}/{epochs}], Average Loss: {total_loss / len(dataloader):.4f}")

In [ ]:
train(model=model, optimizer=optimizer, dataloader=dataloader, epochs=8)


In [ ]:
import torch
import os

# Save the model to the output directory
output_dir = '/kaggle/working/output/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save the model state dictionary
torch.save(model.state_dict(), os.path.join(output_dir, 'trained_model.pth'))

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model architecture and tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the saved model state dictionary
model_path = '/kaggle/working/output/trained_model.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)


In [ ]:
def topk(probs, n=9):
    # The scores are initially softmaxed to convert to probabilities
    probs = torch.softmax(probs, dim= -1)
    
    # PyTorch has its own topk method, which we use here
    tokensProb, topIx = torch.topk(probs, k=n)
    
    # The new selection pool (9 choices) is normalized
    tokensProb = tokensProb / torch.sum(tokensProb)

    # Send to CPU for numpy handling
    tokensProb = tokensProb.cpu().detach().numpy()

    # Make a random choice from the pool based on the new prob distribution
    choice = np.random.choice(n, 1, p = tokensProb)
    tokenId = topIx[choice][0]

    return int(tokenId)

In [ ]:
def model_infer(model, tokenizer, review, max_length=15):
    # Preprocess the init token (task designator)
    review_encoded = tokenizer.encode(review)
    result = review_encoded
    initial_input = torch.tensor(review_encoded).unsqueeze(0).to(device)

    with torch.set_grad_enabled(False):
        # Feed the init token to the model
        output = model(initial_input)

        # Flatten the logits at the final time step
        logits = output.logits[0,-1]

        # Make a top-k choice and append to the result
        result.append(topk(logits))

        # For max_length times:
        for _ in range(max_length):
            # Feed the current sequence to the model and make a choice
            input = torch.tensor(result).unsqueeze(0).to(device)
            output = model(input)
            logits = output.logits[0,-1]
            res_id = topk(logits)

            # If the chosen token is EOS, return the result
            if res_id == tokenizer.eos_token_id:
                return tokenizer.decode(result)
            else: # Append to the sequence 
                result.append(res_id)
    # IF no EOS is generated, return after the max_len
    return tokenizer.decode(result)

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

def calculate_rouge_scores(original_summary, predicted_summary):
    # Initialize Rouge
    rouge = Rouge()

    # Calculate ROUGE scores
    scores = rouge.get_scores(predicted_summary, original_summary)

    # Print ROUGE scores
    print("ROUGE-1: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(
        scores[0]['rouge-1']['p'], scores[0]['rouge-1']['r'], scores[0]['rouge-1']['f']))
    print("ROUGE-2: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(
        scores[0]['rouge-2']['p'], scores[0]['rouge-2']['r'], scores[0]['rouge-2']['f']))
    print("ROUGE-L: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(
        scores[0]['rouge-l']['p'], scores[0]['rouge-l']['r'], scores[0]['rouge-l']['f']))

In [ ]:
while True:
    print("\nEnter your review (type 'exit' to quit):")
    review = input()
    if review.lower() == 'exit':
        break

    print("Enter your summary prompt (press Enter for default ' TL;DR '):")
    summary_prompt = input().strip()
    if not summary_prompt:
        summary_prompt = " TL;DR "

    generated_summary = model_infer(model, tokenizer, review + summary_prompt +" TL;DR ").split(" TL;DR ")[1].strip()
    print("\nGenerated Summary:", generated_summary)
    calculate_rouge_scores(summary_prompt, generated_summary)